# Tarea 2 Ingeniería Financiera

Este Notebook contiene el análisis completo seccionado por pregunta.

In [8]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
import pandas as pd
import numpy as np

In [9]:
### Setup: Lectura de datos
# Leer todas las hojas del archivo Excel
ruta_archivo = "Base de datos completa.xlsx"  # Ajustar ruta si es necesario
datos = pd.read_excel(ruta_archivo, sheet_name=None)

In [10]:
# Mostrar las primeras filas de cada hoja
for hoja, df in datos.items():
    print(f"Hoja: {hoja}")
    print(df.head())

Hoja: Tipo de Cambio
        Date   Price
0 2022-04-25  834.45
1 2022-04-26  850.85
2 2022-04-27  847.57
3 2022-04-28  847.44
4 2022-04-29  856.58
Hoja: Apple Stock Price History (3)
        Date   Price
0 2022-04-25  162.88
1 2022-04-26  156.80
2 2022-04-27  156.57
3 2022-04-28  163.64
4 2022-04-29  157.65
Hoja: Coca-Cola Stock Price History (
        Date  Price
0 2022-04-25  65.94
1 2022-04-26  65.05
2 2022-04-27  65.56
3 2022-04-28  66.19
4 2022-04-29  64.61
Hoja: Banco De Chile Stock Price Hist
        Date  Price
0 2022-04-25  20.95
1 2022-04-26  20.67
2 2022-04-27  20.60
3 2022-04-28  20.14
4 2022-04-29  20.08
Hoja: Cervecerias ADR Stock Price His
        Date  Price
0 2022-04-25  13.40
1 2022-04-26  13.34
2 2022-04-27  13.37
3 2022-04-28  13.37
4 2022-04-29  13.34
Hoja: Enel Chile ADR Stock Price Hist
        Date  Price
0 2022-04-25   1.42
1 2022-04-26   1.36
2 2022-04-27   1.33
3 2022-04-28   1.36
4 2022-04-29   1.34
Hoja: LATAM Airlines ADR Stock Price 
        Date    Price

In [16]:
### Procesamiento de datos
# Calcular retornos logarítmicos diarios excluyendo el último año (solo para hojas con 'Date' y 'Price'; se omite 'Tipo de Cambio')
log_returns = {}
exchange_rate = datos.get("Tipo de Cambio")
if exchange_rate is not None and not exchange_rate.empty:
    exchange_rate['Date'] = pd.to_datetime(exchange_rate['Date'])
    exchange_rate.sort_values('Date', inplace=True)
    # Se asume que la tasa se encuentra en la columna 'Price' de la hoja 'Tipo de Cambio'
for asset, df in datos.items():
    if asset == "Tipo de Cambio":
        print(f"Sheet {asset} skipped because it's only exchange rate")
        continue  # Omitir esta hoja
    if df.empty or 'Date' not in df.columns or 'Price' not in df.columns:
        print(f"Sheet {asset} skipped, missing 'Date' or 'Price' columns or is empty.")
        continue
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    # Convertir precio a pesos usando la tasa de cambio según la fecha
    if exchange_rate is not None and not exchange_rate.empty:
        # Merge por fecha; se asume que las fechas coinciden
        df = pd.merge(df, exchange_rate[['Date', 'Price']], on='Date', how='left', suffixes=('', '_tc'))
        # Calcular el precio en pesos; 'Price_tc' es la tasa de cambio
        df['Price'] = df['Price'] * df['Price_tc']
    cutoff = df['Date'].max() - pd.DateOffset(years=1)
    df_filtered = df[df['Date'] <= cutoff].copy()
    df_filtered['log_return'] = np.log(df_filtered['Price'] / df_filtered['Price'].shift(1))
    log_returns[asset] = df_filtered.dropna(subset=['log_return'])


Sheet Tipo de Cambio skipped because it's only exchange rate


In [17]:
# Calcular retornos y volatilidades anuales
expected_returns_annual = {}
volatility_annual = {}
for asset, df in log_returns.items():
    mu_daily = df['log_return'].mean()
    sigma_daily = df['log_return'].std()
    expected_returns_annual[asset] = mu_daily * 252
    volatility_annual[asset] = sigma_daily * np.sqrt(252)

In [18]:
# Unir los retornos en un DataFrame (excluyendo la hoja 'Tipo de Cambio') y calcular la matriz de covarianza anualizada
returns_df = pd.DataFrame({asset: df.set_index('Date')['log_return'] for asset, df in log_returns.items() if asset != "Tipo de Cambio"})
cov_daily = returns_df.cov()
cov_annual = cov_daily * 252

### (a) Retornos esperados y volatilidades
print("Retornos esperados anuales por activo:", expected_returns_annual)
print("Volatilidades anuales por activo:", volatility_annual)

Retornos esperados anuales por activo: {'Apple Stock Price History (3)': 0.06423503148322413, 'Coca-Cola Stock Price History (': -0.009920298981633935, 'Banco De Chile Stock Price Hist': 0.08066535806790401, 'Cervecerias ADR Stock Price His': 0.032947083554400104, 'Enel Chile ADR Stock Price Hist': 0.3765000559882921, 'LATAM Airlines ADR Stock Price ': 0.7099133123279309}
Volatilidades anuales por activo: {'Apple Stock Price History (3)': 0.31701069894139905, 'Coca-Cola Stock Price History (': 0.22709473341007225, 'Banco De Chile Stock Price Hist': 0.3153608982243638, 'Cervecerias ADR Stock Price His': 0.3384033411492278, 'Enel Chile ADR Stock Price Hist': 0.4848994954948085, 'LATAM Airlines ADR Stock Price ': 1.0757716077798785}


## (b) Cartera de mínima varianza y (c) Recomendación: Cartera de mínimo riesgo

La siguiente celda estima la cartera de mínima varianza (parte de la frontera eficiente) y emite una recomendación.

In [ ]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
## Cartera de mínima varianza
ones = np.ones(len(returns_df.columns))
inv_cov = np.linalg.inv(cov_annual)
w_min = inv_cov.dot(ones) / ones.dot(inv_cov).dot(ones)
port_return_min = sum(w_min[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns))
port_vol_min = np.sqrt(w_min.T.dot(cov_annual).dot(w_min))
print("Cartera de mínima varianza:")
print("Pesos:", w_min)
print("Retorno anual esperado:", port_return_min)
print("Volatilidad anual:", port_vol_min)

## (c) Recomendación de inversión: usar la cartera de mínimo riesgo
print("Recomendación de inversión (cartera de mínimo riesgo):")
print("Pesos:", w_min)
print("Rentabilidad esperada anual:", port_return_min)
print("Volatilidad anual:", port_vol_min)

## (d) Cartera tangente con tasa libre de riesgo

Se utiliza una tasa libre de riesgo (por ejemplo, 3%) para obtener la cartera tangente.

In [ ]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
risk_free_rate = 0.03
excess_returns = np.array([expected_returns_annual[asset] - risk_free_rate for asset in returns_df.columns])
w_tan = inv_cov.dot(excess_returns) / ones.dot(inv_cov).dot(excess_returns)
tan_return = sum(w_tan[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns))
tan_vol = np.sqrt(w_tan.T.dot(cov_annual).dot(w_tan))
print("Cartera tangente (nueva frontera eficiente):")
print("Pesos:", w_tan)
print("Retorno anual esperado:", tan_return)
print("Volatilidad anual:", tan_vol)

## (e) Cartera de mercado y betas

Se asume que la cartera de mercado es la cartera tangente calculada en (d). Se calcula el retorno y la volatilidad anual, y se estima el beta de cada activo.

In [ ]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
market_weights = w_tan
market_return_daily = returns_df.dot(market_weights)
market_return_expected = market_return_daily.mean() * 252
market_vol_daily = market_return_daily.std()
market_vol_annual = market_vol_daily * np.sqrt(252)
print("Cartera de mercado (tangente):")
print("Pesos:", market_weights)
print("Retorno anual esperado:", market_return_expected)
print("Volatilidad anual:", market_vol_annual)

market_var_daily = market_return_daily.var()
betas = {}
for asset in returns_df.columns:
    cov_asset = np.cov(returns_df[asset], market_return_daily)[0, 1]
    betas[asset] = cov_asset / market_var_daily
print("Betas de cada activo respecto a la cartera de mercado:", betas)

## (f) Optimización sin ventas cortas

Utilizando `scipy.optimize.minimize`, se optimiza la cartera de mínima varianza y la tangente imponiendo que todos los pesos sean mayores o iguales a cero y que sumen 1.

In [ ]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
from scipy.optimize import minimize

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(weights.T.dot(cov_matrix).dot(weights))

n = len(returns_df.columns)
bounds = [(0, 1)] * n
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# f.1) Cartera de mínima varianza sin ventas cortas
def min_vol_objective(weights):
    return portfolio_volatility(weights, cov_annual)

res_min_ns = minimize(min_vol_objective, np.ones(n)/n, bounds=bounds, constraints=constraints)
w_min_ns = res_min_ns.x
port_return_min_ns = sum(w_min_ns[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns))
port_vol_min_ns = portfolio_volatility(w_min_ns, cov_annual)
print("Cartera de mínima varianza (sin ventas cortas):")
print("Pesos:", w_min_ns)
print("Retorno anual esperado:", port_return_min_ns)
print("Volatilidad anual:", port_vol_min_ns)

# f.2) Cartera tangente sin ventas cortas (maximizar Sharpe)
def neg_sharpe(weights):
    port_return = sum(weights[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns))
    port_vol = portfolio_volatility(weights, cov_annual)
    sharpe = (port_return - risk_free_rate) / port_vol if port_vol != 0 else -1e6
    return -sharpe

res_tan_ns = minimize(neg_sharpe, np.ones(n)/n, bounds=bounds, constraints=constraints)
w_tan_ns = res_tan_ns.x
tan_return_ns = sum(w_tan_ns[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns))
tan_vol_ns = portfolio_volatility(w_tan_ns, cov_annual)
print("Cartera tangente (sin ventas cortas):")
print("Pesos:", w_tan_ns)
print("Retorno anual esperado:", tan_return_ns)
print("Volatilidad anual:", tan_vol_ns)

## (g) Construcción de 3 carteras en la frontera sin ventas cortas y evaluación del desempeño

Se crean tres portafolios (A, B y C) con distintos objetivos de retorno y se evalúa el desempeño en el último año.

In [ ]:
# filepath: /Users/leandrovenegas/Documents/GitHub/Uni/tarea2.ipynb
def optimize_portfolio_target(target_return):
    cons = constraints + [{'type': 'eq', 'fun': lambda w: sum(w[i] * expected_returns_annual[asset] for i, asset in enumerate(returns_df.columns)) - target_return}]
    res = minimize(min_vol_objective, np.ones(n)/n, bounds=bounds, constraints=cons)
    return res.x

# Definir 3 objetivos de retorno
target_low = port_return_min_ns + 0.0005   # objetivo ligeramente superior a la cartera mínima
target_med = (port_return_min_ns + tan_return_ns) / 2
target_high = tan_return_ns - 0.0005          # objetivo ligeramente inferior al máximo

w_low = optimize_portfolio_target(target_low)
w_med = optimize_portfolio_target(target_med)
w_high = optimize_portfolio_target(target_high)

def performance_last_year(weights):
    last_year_returns = []
    for asset, df in datos.items():
        if df.empty or 'Date' not in df.columns or 'Price' not in df.columns:
            continue
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values('Date', inplace=True)
        cutoff = df['Date'].max() - pd.DateOffset(years=1)
        df_ly = df[df['Date'] > cutoff].copy()
        df_ly['log_return'] = np.log(df_ly['Price'] / df_ly['Price'].shift(1))
        df_ly = df_ly.dropna(subset=['log_return'])
        last_year_returns.append(df_ly.set_index('Date')['log_return'].rename(asset))
    if not last_year_returns:
        return None, None
    ly_df = pd.concat(last_year_returns, axis=1).dropna()
    port_daily = ly_df.dot(weights)
    cum_return = np.exp(port_daily.sum()) - 1
    return port_daily.mean(), cum_return

print("Cartera A (alto riesgo, sin ventas cortas):")
print("Pesos:", w_high)
mean_A, cum_A = performance_last_year(w_high)
print("Retorno diario promedio:", mean_A, "Retorno acumulado:", cum_A)

print("Cartera B (riesgo mediano, sin ventas cortas):")
print("Pesos:", w_med)
mean_B, cum_B = performance_last_year(w_med)
print("Retorno diario promedio:", mean_B, "Retorno acumulado:", cum_B)

print("Cartera C (bajo riesgo, sin ventas cortas):")
print("Pesos:", w_low)
mean_C, cum_C = performance_last_year(w_low)
print("Retorno diario promedio:", mean_C, "Retorno acumulado:", cum_C)

## (h) Modelo Black-Litterman (Bonus)

Implementación simplificada del modelo Black-Litterman. Se parte de los retornos implícitos del mercado y se incorpora una vista sobre el primer activo.